<a href="https://colab.research.google.com/github/Code-by-Mann/Guardian-Voice-AI/blob/main/Guardian_Voice_Capstone_safety_agent_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import json
import time
from datetime import datetime

# In a real deployment, use: import google.generativeai as genai
# For this capstone demonstration, we simulate the LLM's agentic behavior
# to ensure the logic is clear without needing an active API key immediately.

class GuardianTools:
    """
    TOOLS: Demonstrating 'Tools' concept from Image 2.
    These allow the Agent to interact with the outside world.
    """

    @staticmethod
    def get_user_context(user_id):
        """Simulates fetching Long-term Memory (Profile) from Firestore."""
        # This is 'Sessions & Memory'
        return {
            "user_id": user_id,
            "name": "Priya Sharma",
            "age": 24,
            "blood_group": "B+",
            "govt_id": "Aadhaar XXXX-XXXX-1234",
            "photo_url": "https://storage.googleapis.com/guardian-voice/priya.jpg",
            "emergency_contacts": ["+919876543210", "+911122334455"],
            "medical_conditions": "Asthma"
        }

    @staticmethod
    def get_live_location():
        """Simulates 'Context Engineering' by providing dynamic state."""
        return {
            "lat": 28.6139,
            "lng": 77.2090,
            "address": "Connaught Place, New Delhi, India",
            "timestamp": datetime.now().isoformat()
        }

    @staticmethod
    def search_nearby_services(service_type, location):
        """Simulates a Search Tool (e.g., Google Places API)."""
        print(f"   [Tool] Searching for nearest '{service_type}' near {location['address']}...")
        return {
            "name": "Connaught Place Police Station",
            "distance": "0.5 km",
            "contact": "100"
        }

    @staticmethod
    def dispatch_alert(message, recipients):
        """Simulates the final Action."""
        print(f"   [Action] 🚨 DISPATCHING ALERT TO: {recipients}")
        print(f"   [Action] 📩 MESSAGE BODY: \n{message}")
        return True

class MultiAgentSystem:
    """
    MULTI-AGENT SYSTEM: Demonstrating the core requirement.
    """

    def __init__(self):
        self.tools = GuardianTools()
        print("--- Guardian Voice System Initialized ---\n")

    def triage_agent(self, transcript):
        """
        Agent 1: The Router.
        Decides WHICH specialist agent handles the request.
        """
        print(f"🤖 [Triage Agent] Analyzing input: '{transcript}'")

        # In real implementation, Gemini 1.5 Flash classifies this.
        # Simulating classification logic:
        transcript = transcript.lower()
        if "help" in transcript or "danger" in transcript or "save" in transcript:
            return "EMERGENCY"
        elif "ride" in transcript or "cab" in transcript:
            return "RIDE_BOOKING"
        elif "report" in transcript or "safe" in transcript:
            return "COMMUNITY_REPORT"
        else:
            return "UNKNOWN"

    def emergency_agent(self, user_id, transcript):
        """
        Agent 2: The Emergency Specialist.
        Uses Context + Tools to craft the perfect rescue packet.
        """
        print("   ⚡ [Emergency Agent] Activated. Gathering context...")

        # 1. Fetch Memory (User Profile)
        user_profile = self.tools.get_user_context(user_id)
        print(f"   [Memory] Loaded profile for: {user_profile['name']}")

        # 2. Fetch Live Context (Location)
        location = self.tools.get_live_location()

        # 3. Use Tool (Search for Police)
        nearest_police = self.tools.search_nearby_services("Police Station", location)

        # 4. Generate Alert Content (Context Engineering)
        # We inject all this data into the prompt for the LLM to draft the message.
        alert_message = (
            f"🆘 SOS ALERT: {user_profile['name']} is in DANGER.\n"
            f"📍 Location: {location['address']} (Lat: {location['lat']}, Lng: {location['lng']})\n"
            f"🏥 Medical: {user_profile['blood_group']}, {user_profile['medical_conditions']}\n"
            f"🆔 Govt ID: {user_profile['govt_id']}\n"
            f"👮 Nearest Help: {nearest_police['name']} ({nearest_police['distance']})\n"
            f"📸 Verification Photo: {user_profile['photo_url']}\n"
            f"🗣️ Transcript: '{transcript}'"
        )

        # 5. Execute Action
        recipients = user_profile['emergency_contacts'] + [nearest_police['contact']]
        self.tools.dispatch_alert(alert_message, recipients)
        return "Alert Sent"

    def run(self, user_input):
        """Main Loop"""
        intent = self.triage_agent(user_input)

        if intent == "EMERGENCY":
            self.emergency_agent("user_123", user_input)
        elif intent == "RIDE_BOOKING":
            print("   🚖 [Ride Agent] Routing to verified female driver system...")
        elif intent == "COMMUNITY_REPORT":
            print("   📍 [Report Agent] Logging safety score to Firestore...")
        else:
            print("   ❓ [System] Could not determine intent.")

# --- CAPSTONE DEMONSTRATION ---
if __name__ == "__main__":
    # Simulate a user triggering the safe word in a sentence
    system = MultiAgentSystem()

    # Test Case 1: Emergency
    user_voice_transcript = "I am at the metro station and someone is following me, please save me!"
    system.run(user_voice_transcript)

    print("\n" + "="*30 + "\n")

    # Test Case 2: Safe Ride
    user_voice_transcript = "I need a safe ride home."
    system.run(user_voice_transcript)

--- Guardian Voice System Initialized ---

🤖 [Triage Agent] Analyzing input: 'I am at the metro station and someone is following me, please save me!'
   ⚡ [Emergency Agent] Activated. Gathering context...
   [Memory] Loaded profile for: Priya Sharma
   [Tool] Searching for nearest 'Police Station' near Connaught Place, New Delhi, India...
   [Action] 🚨 DISPATCHING ALERT TO: ['+919876543210', '+911122334455', '100']
   [Action] 📩 MESSAGE BODY: 
🆘 SOS ALERT: Priya Sharma is in DANGER.
📍 Location: Connaught Place, New Delhi, India (Lat: 28.6139, Lng: 77.209)
🏥 Medical: B+, Asthma
🆔 Govt ID: Aadhaar XXXX-XXXX-1234
👮 Nearest Help: Connaught Place Police Station (0.5 km)
📸 Verification Photo: https://storage.googleapis.com/guardian-voice/priya.jpg
🗣️ Transcript: 'I am at the metro station and someone is following me, please save me!'


🤖 [Triage Agent] Analyzing input: 'I need a safe ride home.'
   🚖 [Ride Agent] Routing to verified female driver system...
